In [1]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
#import requests #request+bs4 조합만으로도 crawling가능
from selenium.webdriver.common.keys import Keys
import time
#import re
#import pandas as pd
import getpass
import copy


In [2]:
#base_url = 'https://everytime.kr/382452' #에타 조교 게시판 base_url
#base_url = 'https://everytime.kr/375372/text/이중' #에타 이중전공/전과 게시판에서 '이중'으로 검색
base_url = 'https://everytime.kr'
login_url = 'https://everytime.kr/login' #에타 들어가려면 로그인을 해야 함

In [3]:
#driver 실행
def drive(url):
    driver = webdriver.Chrome('./chromedriver') #driver 객체 불러옴
    driver.implicitly_wait(3) # 3초 후에 작동하도록
    driver.get(url) #url에 접속
    html = driver.page_source #현재 driver에 나타난 창의 page_source(html) 가져오기
    soup = BeautifulSoup(html, 'html.parser') #html 파싱(parsing)을 위해 BeautifulSoup에 넘겨주기
    return driver, soup

In [1]:
#login
def login():
    login_url = 'https://everytime.kr/login'
    driver, _ = drive(login_url)
    _id = input('id: ')
    _pw = getpass.getpass()
    driver.find_element_by_name('userid').send_keys(_id)
    driver.find_element_by_name('password').send_keys(_pw)
    driver.find_element_by_xpath('//*[@id="container"]/form/p[3]/input').click()
    return driver

#qt를 이용해서 login할 때, 인자를 따로 받아줌
def login_qt(_id, _pw):
    login_url = 'https://everytime.kr/login'
    driver, _ = drive(login_url)
    driver.find_element_by_name('userid').send_keys(_id)
    driver.find_element_by_name('password').send_keys(_pw)
    driver.find_element_by_xpath('//*[@id="container"]/form/p[3]/input').click()
    return driver

In [5]:
#get soup, 해당 페이지의 soup만 가져오는 함수
def get_soup(driver):
    body = driver.find_element_by_tag_name('body')
    num_page_down = 1
    while num_page_down:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1.5)
        num_page_down -= 1
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    return soup



In [6]:
def get_board_names():
    driver = login()
    soup = get_soup(driver)
    driver.close()
    board = []
    board_tags = soup.select("#submenu")[0].select('a')

    for b in board_tags:
        board.append((b.text,b.get('data-id')))
        board_dic = dict(board)
    return board_dic

def save_board_names(board_dic):
    data = '' #여기에 게시판이름 \t 게시판id \n 식으로 텍스트를 넣을 것임.
    b_list = list(board_dic)
    for i in b_list:
        try:
            data += i + '\t' + board_dic[i] + '\n' #데이터 입력
        except:
            continue #게시판id에 None type이 올 수도 있기에 이 경우에는 그냥 넘긴다. (게시판 찾기의 경우임)
    f = open('게시판명.txt', 'w', encoding = 'utf8')
    f.write(data)
    print('파일이 작성되었습니다.')
    f.close()
    
def load_board_names(file_name):
    f = open(file_name, 'r', encoding = 'utf8')
    data = f.read()
    board_list = data.split('\n')
    b_tuple_list = []
    for b in board_list:
        try:
            b_tuple_list.append((b.split('\t')[0], b.split('\t')[1])) #마지막 줄은 ''이어서 split 후 index로 참조 못하기에 try문으로 넘어가도록 한다. 
        except:
            continue
       
    board_dic = dict(b_tuple_list)
    f.close()
    return board_dic

def board_selection(board_dic, board_name): #게시판 선택
    global base_url
    base_url = base_url + '/' +board_dic[board_name]
    
def board_query(board_dic, query): #게시판 내 질문 선택
    global base_url
    base_url = base_url + '/all/' +query
    

board_dic = get_board_names()

save_board_names(board_dic)

board_dic = load_board_names('게시판명.txt')

board_dic

# 게시판 글 정보 크롤링 함수, 글과 댓글 크롤링 위한 기초 작업

In [17]:
class counsel_data:
    def __init__(self, driver):
        global base_url
        #self.base_url = 'https://everytime.kr/382452'
        self.writer_list = []
        self.more_list = []
        self.content_list = []
        self.id_list = []
        self.time_list = []
        self.vote_list = []
        self.comment_list = []
        self.driver = driver
        self.driver.get(base_url)
        
        
    def check_rep(self, id_list):
        rep_idx = []
        for i in id_list:
            if i in self.id_list:
                rep_idx.append(id_list.index(i))
        return rep_idx
    
    def get_soup(self):
        body = self.driver.find_element_by_tag_name('body')
        num_page_down = 1
        
        while num_page_down:
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(1.5)
            num_page_down -= 1
            
        html = self.driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        return soup
    
    def get_counsel_data(self):
        soup = self.get_soup()

        counsel = soup.select('.wrap.articles')
        #글쓴이
        writer_list = list(map(lambda x: x.text, counsel[0].select('h3')))

        #더보기 여부
        temp_list = counsel[0].select('a[class="article"]')
        more_list = []
        for i in temp_list:
            if i.select('.more'):
                more_list.append(1)
            else:
                more_list.append(0)
        #글 내용
        content_list = list(map(lambda x: x.text, counsel[0].select('p')))

        #글 접근 위한 id주소
        id_list = list(map(lambda x: x.get('name').split('_')[0], counsel[0].select('input[name]')))
        if len(id_list)>20:
            id_list.pop()
            

        #글 작성 시간
        time_list = list(map(lambda x: x.text, counsel[0].select('time')))

        #좋아요 수
        vote_list = list(map(lambda x: x.text, counsel[0].select('.vote')))

        #댓글 수
        comment_list = list(map(lambda x: x.text, counsel[0].select('.comment')))
        
        
        
        return writer_list, more_list, content_list, id_list, time_list, vote_list, comment_list
    
    #크롤링 시작
    def start_crawling(self):
        start_idx = 1
        writer_list, more_list, content_list, id_list, time_list, vote_list, comment_list = self.get_counsel_data()
        self.writer_list += writer_list
        self.more_list += more_list
        self.content_list += content_list
        self.id_list += id_list
        self.time_list += time_list
        self.vote_list += vote_list
        self.comment_list += comment_list
        
        while True:
            start_idx+=1
            if start_idx%10 ==0:
                print(start_idx)
            self.driver.get(base_url+f'/p/{start_idx}')
            try:
                writer_list, more_list, content_list, id_list, time_list, vote_list, comment_list = self.get_counsel_data()
                
                self.writer_list += writer_list
                self.more_list += more_list
                self.content_list += content_list
                self.id_list += id_list
                self.time_list += time_list
                self.vote_list += vote_list
                self.comment_list += comment_list
            except:
                print('finished')
                self.driver.close()
                break
            if not writer_list: #가져 온 데이터가 아무 것도 없으면 page 색인 종료
                print('finished')
                break
            
        

## 게시판 고르고 로그인 후 class 생성
board_dic = load_board_names('게시판명.txt')

for b in board_dic:
    print(b)
    
board_name = input('크롤링하고 싶은 게시판명을 골라주세요: ')
board_selection(board_dic, board_name)

driver = login()
cns_data = counsel_data(driver)

## 크롤링
cns_data.start_crawling()

# 댓글 크롤링 함수

In [31]:
def get_comments(driver, ids):
    ids = ids
    #반환 값이 담길 리스트들
    articles = []
    comments = []
    comments_writers = [] 
    comments_tag = []
    
    max_len = len(ids)
    proceed = -1
    
    for n,i in enumerate(ids):
        percentage = int((n/max_len)*100)
        if percentage%10 == 0 and percentage>proceed:
            print(percentage)
            proceed = percentage
            time.sleep(3)
            
        driver.get(base_url+f'/v/{i}')
        soup = get_soup(driver)
        
        page_info = soup.select('.wrap.articles')[0]
        
        temp_art = list(map(lambda x: x.text, page_info.select('.article')[0].select('p')))
        articles+=temp_art
        
        
        #댓글 정보들
        comments_info = page_info.select('.comments')[0]
        
        #댓글 
        temp_comments = list(map(lambda x: x.text, comments_info.select('.large')))
        if temp_comments: #댓글이 있으면 추가한다.
            comments.append(temp_comments)
        else:
            comments+=[None]
        
        #댓글 작성자
        temp_comments_writers = list(map(lambda x: x.text, comments_info.select('h3')))
        if comments_writers:
            comments_writers.append(temp_comments_writers)
        else:
            comments_writers += [None]
                
        # 댓글 대댓글 여부
        if comments_info.select('article'):
            temp_comments_tag = list(map(lambda x: x.get('class')[0], comments_info.select('article')))
            comments_tag.append(temp_comments_tag)
            
        else:
            #댓글이 없으면 그냥 None 넣어줌
            
            comments_tag += [None]
        
        
    return articles, comments,  comments_writers, comments_tag, ids

len(cns_data.id_list)

# 크롤러 실행

In [35]:
#crawling 1000개씩 나누기
def crawl_devision(cns_data):
    ids = copy.deepcopy(cns_data.id_list)
    n = len(ids)//1000 #n*1000개의 data, 1000보다 작으면 그냥 바로 계산
    if n<1: #1000개보다 적다.
        return 0, [ids] #0은 n값
    else: 
        ids_n = []
        for i in range(n+1):
            if i == n: #마지막 자투리
                ids_n.append(ids[:len(ids)])
            else:
                ids_n.append(ids[i*1000:(i+1)*1000])
        return n, ids_n

def crawl_ntimes(n, ids):
    n = n+1 #n=0이면 1번 크롤링, n=1이면 2번 크롤링...
    articles = []
    comments = []
    comments_writers = []
    comments_tag = []
    ids_result = []
    
    for i in range(n):
        driver = login()
        articles_t, comments_t,  comments_writers_t, comments_tag_t, ids_t = get_comments(driver, ids[i])
        print(i)
        articles += articles_t
        comments += comments_t
        comments_writers += comments_writers_t
        comments_tag += comments_tag_t
        ids_result += ids_t
            
    
    driver.close()
    return articles, comments, comments_writers, comments_tag, ids_result


n, ids = crawl_devision(cns_data)

articles, comments, comments_writers, comments_tag, ids_result = crawl_ntimes(n, ids)

## 미리 크롤링한 id들을 deepcopy

ids = copy.deepcopy(cns_data.id_list)

## 추출할 양이 적으면 그냥 실행(1000개 이하)

ids4 = ids[:len(cns_data.id_list)]

driver = login()

articles, comments,  comments_writers, comments_tag, ids = get_comments(driver, ids)

print(len(articles))
print(len(comments))
print(len(comments_writers))
print(len(comments_tag))
print(len(ids))

from pandas import Series, DataFrame

raw_data = {'내용': articles,
            '댓글': comments,
           '댓글쓴이':comments_writers,
           '대댓글여부':comments_tag,
           'id':ids_result,}

data = DataFrame(raw_data)
data[30:40]

## csv로 저장

data.to_csv("이중전과게시판_키워드_이중.csv", mode='w', encoding='utf-8')